In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import urllib.request
import re

In [3]:
url = "https://www.gutenberg.org/cache/epub/6130/pg6130.txt"
file = urllib.request.urlopen(url)
text = [line.decode('utf-8') for line in file]
text = ''.join(text)
text = re.sub(' +',' ',text)
text = re.sub(r'[^A-Za-z.,!\r ]+', '', text)
text = text[1150:]
text[:200]

'X.\r BOOK XXI.\r BOOK XXII.\r BOOK XXIII.\r BOOK XXIV.\r\r CONCLUDING NOTE.\r\r\r\r\rIllustrations\r\r HOMER INVOKING THE MUSE\r MARS\r MINERVA REPRESSING THE FURY OF ACHILLES\r THE DEPARTURE OF BRISEIS FROM THE TENT'

In [2]:
import pandas as pd

In [6]:
df_messages = pd.read_csv("out.csv")
df_messages.sample(7)

,Unnamed: 0,_id,time,channel,author,test
12175,12175,655cc316b0a0a07b2eba608a,17:47:50,pwgood,Mitsuka_Yumi,yaderka 󠀀
9589,9589,655cb677b0a0a07b2eba5670,16:53:59,pwgood,androtoria,*****
18024,18024,655ce26fb0a0a07b2eba7763,20:01:35,pwgood,Elariyan,Pepepains PianoTime
5029,5029,655ca429b0a0a07b2eba44a0,15:35:53,pwgood,tenno_kory,"@avacuoss, re"
14581,14581,655cd05cb0a0a07b2eba69f0,18:44:28,pwgood,Kabe1ka,что за раннер?
6233,6233,655ca82cb0a0a07b2eba4954,15:53:00,pwgood,fengsheesh,AlienPls 󠀀
12365,12365,655cc387b0a0a07b2eba6148,17:49:43,pwgood,MbIshka_PupbIshka_,catSleep


In [7]:
text = ' '.join(list(df_messages['test']))

In [8]:
text

'так так так JABA ping Alright $cookie @likozap woke up: GoslingDrive 💤 (9h, 56m ago) likozap, Your daily cookie: An opportunity to do some extra work might come your way today. Don\'t turn it down - it might do you a lot of good. Я в расиииии peepoHappier peepoHappier Втф monka PWGud SomethingHappend? ViolinTime $$ EtoKomu 👥 drt_s_s, Чита, Забайкальский край, Россия (сейчас): 🌤️ -8,18°С, по ощущениям -11,98°С. Облачность: 12%. Скорость юго-западного ветра: 2 м/с. Никаких порывов ветра. Влажность: 52%. Осадков сейчас нет. Давление воздуха: 1017 гПа. Солнце садится через 7 часов 14 минут. $weather hour+3 @kselbot feels rested: Panic Panic 💤 (6h, 35m ago) kselbot, Perm, Perm Krai, Russia (09:00 local time): 🌧️ -5.73°C, feels like -11.98°C. Cloud cover: 90%. ESE wind speed: 4.86 m/s. Wind gusts: up to 12.52 m/s. Humidity: 89%. 100% chance of 0.57mm/h snow. Air pressure: 1017 hPa. ⚠️ Weather alerts: Black ice, Extreme temperature value, Rain, Snow/Ice. Zevlo @jehervs feels rested: catSleep

In [9]:
vocab = sorted(set(text))
",".join(vocab)

'\x01, ,!,",#,$,%,&,\',(,),*,+,,,-,.,/,0,1,2,3,4,5,6,7,8,9,:,;,<,=,>,?,@,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,[,\\,],^,_,`,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,{,|,},~,«,°,·,»,¿,ë,ĺ,˚,́,̶,͜,͞,Ё,А,Б,В,Г,Д,Е,Ж,З,И,Й,К,Л,М,Н,О,П,Р,С,Т,У,Ф,Х,Ц,Ч,Ш,Щ,Ъ,Ы,Ь,Э,Ю,Я,а,б,в,г,д,е,ж,з,и,й,к,л,м,н,о,п,р,с,т,у,ф,х,ц,ч,ш,щ,ъ,ы,ь,э,ю,я,ё,ї,ل,ຈ,༼,༽,\u200d,–,—,―,’,•,…,₽,№,←,→,√,≈,⊹,⌒,⌛,⏰,⏳,ⓘ,☀,☁,☆,☝,☹,♂,♡,♥,⚠,⚡,⛄,⛵,✊,✋,✌,✏,✨,❔,❕,❗,➡,⠀,⠁,⠃,⠄,⠈,⠋,⠙,⠛,⠟,⠰,⠸,⠻,⠿,⡀,⡄,⡇,⡏,⡟,⡿,⢀,⢁,⢋,⢠,⢸,⢿,⣄,⣠,⣤,⣦,⣧,⣬,⣴,⣶,⣷,⣼,⣽,⣾,⣿,Ɑ,〈,ノ,ー,ヽ,彡,王,구,녕,안,친,ﮞ,️,ﻝ,／,＞,Ｙ,＼,＿,｜,､,ﾉ,ﾐ,ﾟ,￣,￼,𝐀,𝐆,𝐏,𝐖,𝐚,𝐛,𝐝,𝐞,𝐟,𝐠,𝐡,𝐢,𝐣,𝐥,𝐨,𝐩,𝐫,𝐬,𝐭,𝐮,🇮,🇱,🇷,🇸,🇺,🌖,🌘,🌙,🌝,🌠,🌤,🌥,🌧,🌫,🌴,🌹,🌿,🍂,🍅,🍇,🍈,🍉,🍌,🍑,🍜,🍧,🍪,🍫,🍬,🍭,🍰,🍵,🍷,🍻,🎂,🎩,🎰,🎼,🏐,🏻,🐁,🐈,🐋,🐙,🐝,🐡,🐭,🐱,🐺,👀,👂,👆,👈,👉,👋,👌,👍,👏,👥,💀,💃,💍,💡,💣,💤,💥,💿,📆,📈,📉,📚,📡,📣,🔙,🔥,🔪,🔮,🕒,🕯,🖕,😃,😉,😋,😍,😎,😏,😑,😒,😘,😧,😨,😭,😮,😱,😲,😶,🙂,🙌,🙏,🚀,🚪,🚫,🚲,🚿,🛎,🛏,🟪,🤓,🤔,🤖,🤘,🤙,🤣,🤧,🤨,🤪,🤯,🥄,🥕,🥗,🥚,🥞,🥯,🥶,🥺,🦅,🦐,🦫,🩴,🪞,🪦,🫖,\U000e0000'

In [10]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
[f"{char} = {i}" for char, i in zip(char2idx, range(20))]

['\x01 = 0',
 '  = 1',
 '! = 2',
 '" = 3',
 '# = 4',
 '$ = 5',
 '% = 6',
 '& = 7',
 "' = 8",
 '( = 9',
 ') = 10',
 '* = 11',
 '+ = 12',
 ', = 13',
 '- = 14',
 '. = 15',
 '/ = 16',
 '0 = 17',
 '1 = 18',
 '2 = 19']

In [54]:
import json

In [56]:
json.dump(vocab, open("NLP_20K_CHAT/vocab.json", 'w'))

In [11]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
[idx2char[i.numpy()] for i in char_dataset.take(5)]

['т', 'а', 'к', ' ', 'т']

In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
[repr(''.join(idx2char[item.numpy()])) for item in sequences.take(5)]

["'так так так JABA ping Alright $cookie @likozap woke up: GoslingDrive 💤 (9h, 56m ago) likozap, Your da'",
 '"ily cookie: An opportunity to do some extra work might come your way today. Don\'t turn it down - it m"',
 "'ight do you a lot of good. Я в расиииии peepoHappier peepoHappier Втф monka PWGud SomethingHappend? V'",
 "'iolinTime $$ EtoKomu 👥 drt_s_s, Чита, Забайкальский край, Россия (сейчас): 🌤️ -8,18°С, по ощущениям -'",
 "'11,98°С. Облачность: 12%. Скорость юго-западного ветра: 2 м/с. Никаких порывов ветра. Влажность: 52%.'"]

In [13]:
@tf.autograph.experimental.do_not_convert
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'так так так JABA ping Alright $cookie @likozap woke up: GoslingDrive 💤 (9h, 56m ago) likozap, Your d'
Target data: 'ак так так JABA ping Alright $cookie @likozap woke up: GoslingDrive 💤 (9h, 56m ago) likozap, Your da'


In [15]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 159 ('т')
  expected output: 141 ('а')
Step    1
  input: 141 ('а')
  expected output: 151 ('к')
Step    2
  input: 151 ('к')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 159 ('т')
Step    4
  input: 159 ('т')
  expected output: 141 ('а')


In [16]:
BATCH_SIZE = 128
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
rnn_units_2 = 512

In [17]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim,
                            batch_input_shape=[BATCH_SIZE, None]),
  tf.keras.layers.GRU(rnn_units,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'), 
  tf.keras.layers.GRU(rnn_units_2,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'),  
  tf.keras.layers.Dense(vocab_size)
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 256)          111616    
                                                                 
 gru (GRU)                   (128, None, 1024)         3938304   
                                                                 
 gru_1 (GRU)                 (128, None, 512)          2362368   
                                                                 
 dense (Dense)               (128, None, 436)          223668    
                                                                 
Total params: 6635956 (25.31 MB)
Trainable params: 6635956 (25.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [19]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [20]:
history = model.fit(dataset, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100



47/47 [==============================] - 138s 3s/step - loss: 4.5315
Epoch 2/100
47/47 [==============================] - 127s 3s/step - loss: 3.4355
Epoch 3/100
47/47 [==============================] - 128s 3s/step - loss: 3.0363
Epoch 4/100
47/47 [==============================] - 135s 3s/step - loss: 2.7867
Epoch 5/100
47/47 [==============================] - 156s 3s/step - loss: 2.5474
Epoch 6/100
47/47 [==============================] - 153s 3s/step - loss: 2.3327
Epoch 7/100
47/47 [==============================] - 141s 3s/step - loss: 2.1433
Epoch 8/100
47/47 [==============================] - 138s 3s/step - loss: 1.9723
Epoch 9/100
47/47 [==============================] - 146s 3s/step - loss: 1.8264
Epoch 10/100
47/47 [==============================] - 136s 3s/step - loss: 1.6979
Epoch 11/100
47/47 [==============================] - 140s 3s/step - loss: 1.5869
Epoch 12/100
47/47 [==============================] - 137s 3s/step - loss: 1.4850
Epoch 13/100
47/47 [================

In [22]:
def generate_text(model, start_string, temp, gen_chars):     
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)  
  text_generated = []
  model.reset_states()
  for i in range(gen_chars):
    predictions = model(input_eval)      
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temp
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])  
  return (start_string + ''.join(text_generated))

In [23]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim,
                            batch_input_shape=[1, None]),
  tf.keras.layers.GRU(rnn_units,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'),
  tf.keras.layers.GRU(rnn_units_2,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'), 
  tf.keras.layers.Dense(vocab_size)
])
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim,
                            batch_input_shape=[1, None]),
  tf.keras.layers.GRU(rnn_units,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'),
  tf.keras.layers.GRU(rnn_units_2,
                      return_sequences=True,
                      stateful=True,
                      recurrent_initializer='glorot_uniform'), 
  tf.keras.layers.Dense(vocab_size)
])
model.summary()
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (1, None, 256)            111616    
                                                                 
 gru_4 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 gru_5 (GRU)                 (1, None, 512)            2362368   
                                                                 
 dense_2 (Dense)             (1, None, 436)            223668    
                                                                 
Total params: 6635956 (25.31 MB)
Trainable params: 6635956 (25.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
generate_text(model, u"ебаться в сраку ", 1, 150)

'ебаться в сраку \U000e0000 NOOOO @PWGood Обед 2 DeneboSO  you ,  to destroy me forsenPls \U000e0000 I need you , to hurt me forsenPls I need you , to fight me forsenPls I need you , to'

In [21]:
model.save("NLP_20K_CHAT")

INFO:tensorflow:Assets written to: NLP_20K_CHAT\assets


INFO:tensorflow:Assets written to: NLP_20K_CHAT\assets


In [53]:
model.save("NLP_20L_CHAT/save_model.h5")

c:\Windows.old\Users\pbrag\AppData\Local\Programs\Python\Python311\pomidor_team_project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
